### Εργασία του Φοιτητή Μυλωνά Αθανασίου ΑΜ: e21118


In [1]:
# Εισαγωγή βιβλιοθηκών
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn as sk

## Εισάγουμε τα δεδομένα για πρώτη φορά


In [2]:
#Φορτώνουμε τα δεδομένα
rawData = pd.read_csv('/home/th4nos/Projects/DataScience/CourseAssignment/Data/data.csv')

# Φτιάχνω ένα αντίγραφο των δεδομένων για λόγους ασφαλείας
data = rawData.copy()
#Εμφανίζουμε τα πρώτα 5 δείγματα
data.head()

,TripId,StartTime,EndTime,StartStationId,EndStationId
0,2616540,2023-02-01 00:28:04,2023-02-01 00:45:18,1,0
1,2616644,2023-02-01 00:34:08,2023-02-01 00:45:49,1,0
2,2616730,2023-02-01 00:39:18,2023-02-01 00:52:06,1,0
3,2617099,2023-02-01 00:59:02,2023-02-01 01:08:31,0,1
4,2617103,2023-02-01 00:59:35,2023-02-01 01:08:19,0,1


### Σαν πρώτη όψη

Το dataset περιλαμβάνει πέντε στήλες: το μοναδικό αναγνωριστικό κάθε διαδρομής (TripId), την ώρα εκκίνησης (StartTime), την ώρα επιστροφής (EndTime), το αναγνωριστικό του σταθμού αφετηρίας (StartStationId) και το αναγνωριστικό του σταθμού προορισμού (EndStationId). Κάθε εγγραφή καταγράφει μια διαδρομή ποδηλάτου με πληροφορίες για τον χρόνο και τους σταθμούς εκκίνησης και προορισμού.


#### Προετοιμασία

Στην συνέχεια θα κάνουμε έναν πρώτο έλεγχο των δεδομένων για να δούμε τι περιέχουν και πως είναι οργανωμένα.


In [3]:
#Εμφανίζουμε τις πληροφορίες των δεδομένων
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8636 entries, 0 to 8635
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   TripId          8636 non-null   int64 
 1   StartTime       8636 non-null   object
 2   EndTime         8636 non-null   object
 3   StartStationId  8636 non-null   int64 
 4   EndStationId    8636 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 337.5+ KB


Με την εμφάνισει των πληροφοριών σχετικά για τι τύπο δεδομένων περιέχει κάθε στήλη, μπορούμε να καταλάβουμε τι είδους επεξεργασία θα χρειαστεί να κάνουμε. Στην συνέχεια θα ελέγξουμε αν υπάρχουν κενές τιμές στο dataset όπως επίσης και αν υπάρχουν διπλές εγγραφές.


In [4]:
# Ελέγχουμε για τυχόν απουσιάζουσες τιμές
data.isnull().sum()

TripId            0
StartTime         0
EndTime           0
StartStationId    0
EndStationId      0
dtype: int64

In [5]:
# Ελέγχουμε για τυχόν διπλές τιμές στο TripId
data.duplicated(subset=['TripId']).sum()

0

In [6]:
# Επίσης ελέγχουμε για το άμα το StartStationId και EndStationId έχουν και τα δύο ίδιες τιμές
dupleCheck = (data['StartStationId'] == data['EndStationId']).all()
dupleCheck

False

Τελικά δεν υπάρχουν κενές τιμές ή διπλές εγγραφές στο dataset.


#### Σαν μια πρώτη αλλάγη θα μετατρέψουμε τις στήλες StartTime και EndTime σε datetime αντικείμενα από object για να μπορούμε να κάνουμε εύκολα υπολογισμούς μεταξύ τους.


In [7]:
# Mετατρέπουμε τις στήλες 'StartTime' και 'EndTime' σε datetime
data['StartTime'] = pd.to_datetime(data['StartTime'])
data['EndTime'] = pd.to_datetime(data['EndTime'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8636 entries, 0 to 8635
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   TripId          8636 non-null   int64         
 1   StartTime       8636 non-null   datetime64[ns]
 2   EndTime         8636 non-null   datetime64[ns]
 3   StartStationId  8636 non-null   int64         
 4   EndStationId    8636 non-null   int64         
dtypes: datetime64[ns](2), int64(3)
memory usage: 337.5 KB


### Στην εκφώνηση της άσκησης μας εξηγεί πως:

**"Η τιμή 1 σε κάποια από τις στήλες StartStationId ή EndStationId υποδηλώνει το σταθμό που μας ενδιαφέρει. Αντίθετα, όπου αναγράφεται η τιμή 0 σε κάποια από τις στήλες αυτές, αυτή υποδηλώνει κάποιον άλλον σταθμό (̸ = 1) που μας είναι αδιάφορος."**

Επομένως θα πρέπει να φιλτράρουμε τα δεδομένα μας ώστε να περιέχουν μόνο τις διαδρομές που μας ενδιαφέρουν.


In [8]:
# Φιλτράρουμε τα δεδομένα με βάση το άμα η StartStationId ή η EndStationId είναι 1
filteredData = data[(data['StartStationId'] == 1) | (data['EndStationId'] == 1)]
filteredData

,TripId,StartTime,EndTime,StartStationId,EndStationId
0,2616540,2023-02-01 00:28:04,2023-02-01 00:45:18,1,0
1,2616644,2023-02-01 00:34:08,2023-02-01 00:45:49,1,0
2,2616730,2023-02-01 00:39:18,2023-02-01 00:52:06,1,0
3,2617099,2023-02-01 00:59:02,2023-02-01 01:08:31,0,1
4,2617103,2023-02-01 00:59:35,2023-02-01 01:08:19,0,1
...,...,...,...,...,...
8631,3988772,2023-02-28 23:29:08,2023-02-28 23:33:32,1,0
8632,3988863,2023-02-28 23:33:01,2023-02-28 23:36:30,0,1
8633,3988901,2023-02-28 23:35:32,2023-02-28 23:40:22,1,0
8634,3988930,2023-02-28 23:36:58,2023-02-28 23:45:17,0,1


#### Για να κάνουμε ανάλυση χρονοσειρών θα χρειαστεί να επιλέξουμε ένα χρονικό διάστημα. Στην συγκεκριμένη περίπτωση θα διαλέξουμε το παράδειγμα που δίνει και στην εκφώνηση που είναι οι **"T = 2 ώρες"**. Αφού ομαδοποιήσουμε τις εκκινήσεις και τις επιστροφές σε κάθε διάστημα **T**, υπολογίζουμε τη ζήτηση ως τη διαφορά μεταξύ των ποδηλάτων που επέστρεψαν και αυτών που παραδόθηκαν εντός του διαστήματος **T**. Με αυτόν τον τρόπο, δημιουργούμε μια χρονοσειρά ζήτησης ποδηλάτων.


In [9]:
# Ορίζουμε το χρονικό διάστημα που θέλουμε να εξετάσουμε
T = '2h'  # Ορισμός του χρονικού διαστήματος σε 2 ώρες

# Δημιουργία της χρονοσειράς για την εκκίνηση από τον ενδιαφερόμενο σταθμό
startSeries = filteredData[filteredData['StartStationId'] == 1].set_index('StartTime').resample(T).count()['TripId']

# Δημιουργία της χρονοσειράς για τον τερματισμό στον ενδιαφερόμενο σταθμό
endSeries = filteredData[filteredData['EndStationId'] == 1].set_index('EndTime').resample(T).count()['TripId']

# Υπολογισμός ζήτησης ποδηλάτων στον ενδιαφερόμενο σταθμό & Δημιουργία της χρονοσειράς
demandSeries = endSeries.sub(startSeries, fill_value=0)

# Εκτύπωση της χρονοσειράς
demandSeries = pd.DataFrame({'Time': demandSeries.index, 'Demand': demandSeries.values})
demandSeries

,Time,Demand
0,2023-02-01 00:00:00,-1
1,2023-02-01 02:00:00,0
2,2023-02-01 04:00:00,1
3,2023-02-01 06:00:00,-10
4,2023-02-01 08:00:00,-7
...,...,...
331,2023-02-28 14:00:00,4
332,2023-02-28 16:00:00,8
333,2023-02-28 18:00:00,6
334,2023-02-28 20:00:00,4


### Χωρισμός των δεδομένων σε train και test set
Θα χωρίσουμε τα δεδομένα σε δύο μέρη: το πρώτο 70% για εκπαίδευση και το υπόλοιπο 30% για έλεγχο.

In [10]:
# Ορίζουμε το ποσοστό των δεδομένων που θα χρησιμοποιήσουμε για το training
trainSize = 0.7

# Υπολογίζουμε τον αριθμό των παρατηρήσεων για εκπαίδευση
trainSize = int(len(demandSeries) * trainSize)

# Διαχωρίζουμε τα δεδομένα σε train και test
trainData = demandSeries[:trainSize]
testData = demandSeries[trainSize:]

### Εφαρμογή μεθόδων πρόβλεψης χρονοσειρών
